In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances

Lx=128 
Ly=128
abun=100
nbirth=ndeath=convert(Int64,floor(abun*0.15))
    
#reps=collect(201:224)
disp=[3,4,6,10]
comp_dist=[3,6]
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(disp,comp_dist,methodrefs[:,:ind]))
lands=[collect(2:6);collect(102:106);collect(201:206)]

result=DataFrame()

for j in 1:length(lands)

landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",lands[j],".csv"), DataFrame)

landscape=landscape[landscape.x .<129 .&& landscape.y .<129 ,:]

repl=landscape.rep[1] 
rangepar=landscape.rangepars[1]    

landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
landscape[:,:rep]=fill(repl,nrow(landscape))
landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

initial=filter(:pres => x-> x .== 1,landscape)
initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
init=Matrix(init[:,Not("y")])

habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
habmat=Matrix{Float64}(habmat[:,Not("y")])
    
#Threshold for a suitable habitat
mid=quantile(landscape[:,:soiltype],0.5)

      
popmat=deepcopy(init)

for i1 in 1:length(pars)
        
    disps=pars[i1][1]
    methods=pars[i1][2]
        
    #Define dispersal probability as a function of #neighbors for all three dispersal methods
    comp_dist=3
    maxnb=((2*comp_dist+1)^2)-1
    neigh=log.(maxnb,1:maxnb)
    comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])    
        
    d=Normal(0,disps/2)
        
    popmat=deepcopy(init)

    for i in 1:200
            
        samp=findall(popmat.==1)
        mort=habmat[samp]
        del=wsample(1:length(mort),mort,ndeath,replace=false)
        popmat[samp[del]].=0
            
        samp2=shuffle(findall(popmat.==0))

        vals=Float64[]

        for i2 in 1:length(samp2)

            x=samp2[i2][1]
            y=samp2[i2][2]


            xs=((x-disps):(x+disps))
            xs=xs[0 .<xs .<=Ly]

            ys=(y-disps):(y+disps)
            ys=ys[0 .<ys .<=Lx]

            prob=0
            
            if sum(popmat[xs,ys])>0      
                    
                parents=findall(popmat[xs,ys].==1)

                for i2 in 1:length(parents)

                    prob+=pdf(d,euclidean([x,y],[xs[parents[i2][1]],ys[parents[i2][2]]]))
                end

                xc=((x-comp_dist):(x+comp_dist))
                xc=xc[0 .<xc .<=Ly]

                yc=(y-comp_dist):(y+comp_dist)
                yc=yc[0 .<yc .<=Lx]
                

                prob*=comp_list[methods][sum(popmat[xc,yc])+1]    

            end

            push!(vals,prob)

        end
    
        popmat[samp2[wsample(1:length(samp2),vals,nbirth,replace=false)]].=1  
        
    end
    
            popmat1=DataFrame(popmat,:auto)
            rename!(x-> strip(string(x), ['x']),popmat1)
            popmat1[:,:y].=1:Ly
            popmat1=stack(popmat1,1:Lx)
            rename!(popmat1,:variable=> :x, :value => :pres)
            popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
            filter!(:pres => x-> x .== 1,popmat1)
            popmat1[:,:rep].=repl 
            popmat1[:,:rangepar].=rangepar
            popmat1[:,:disp].=disps
            popmat1[:,:method].=methodrefs[methods,2]        
            append!(result,popmat1)
        
    
end
end

     

In [1]:
?fill

search: fill fill! finally findall filter filter! filesize filemode isfile



```
fill(value, dims::Tuple)
fill(value, dims...)
```

Create an array of size `dims` with every location set to `value`.

For example, `fill(1.0, (5,5))` returns a 5×5 array of floats, with `1.0` in every location of the array.

The dimension lengths `dims` may be specified as either a tuple or a sequence of arguments. An `N`-length tuple or `N` arguments following the `value` specify an `N`-dimensional array. Thus, a common idiom for creating a zero-dimensional array with its only location set to `x` is `fill(x)`.

Every location of the returned array is set to (and is thus [`===`](@ref) to) the `value` that was passed; this means that if the `value` is itself modified, all elements of the `fill`ed array will reflect that modification because they're *still* that very `value`. This is of no concern with `fill(1.0, (5,5))` as the `value` `1.0` is immutable and cannot itself be modified, but can be unexpected with mutable values like — most commonly — arrays.  For example, `fill([], 3)` places *the very same* empty array in all three locations of the returned vector:

```jldoctest
julia> v = fill([], 3)
3-element Vector{Vector{Any}}:
 []
 []
 []

julia> v[1] === v[2] === v[3]
true

julia> value = v[1]
Any[]

julia> push!(value, 867_5309)
1-element Vector{Any}:
 8675309

julia> v
3-element Vector{Vector{Any}}:
 [8675309]
 [8675309]
 [8675309]
```

To create an array of many independent inner arrays, use a [comprehension](@ref man-comprehensions) instead. This creates a new and distinct array on each iteration of the loop:

```jldoctest
julia> v2 = [[] for _ in 1:3]
3-element Vector{Vector{Any}}:
 []
 []
 []

julia> v2[1] === v2[2] === v2[3]
false

julia> push!(v2[1], 8675309)
1-element Vector{Any}:
 8675309

julia> v2
3-element Vector{Vector{Any}}:
 [8675309]
 []
 []
```

See also: [`fill!`](@ref), [`zeros`](@ref), [`ones`](@ref), [`similar`](@ref).

# Examples

```jldoctest
julia> fill(1.0, (2,3))
2×3 Matrix{Float64}:
 1.0  1.0  1.0
 1.0  1.0  1.0

julia> fill(42)
0-dimensional Array{Int64, 0}:
42

julia> A = fill(zeros(2), 2) # sets both elements to the same [0.0, 0.0] vector
2-element Vector{Vector{Float64}}:
 [0.0, 0.0]
 [0.0, 0.0]

julia> A[1][1] = 42; # modifies the filled value to be [42.0, 0.0]

julia> A # both A[1] and A[2] are the very same vector
2-element Vector{Vector{Float64}}:
 [42.0, 0.0]
 [42.0, 0.0]
```


In [4]:
    items = [item for item in walkdir("C:\\Users\\mihir\\Documents\\landscapes")][1][3]

300-element Vector{String}:
 "landscape_1.csv"
 "landscape_10.csv"
 "landscape_100.csv"
 "landscape_101.csv"
 "landscape_102.csv"
 "landscape_103.csv"
 "landscape_104.csv"
 "landscape_105.csv"
 "landscape_106.csv"
 "landscape_107.csv"
 "landscape_108.csv"
 "landscape_109.csv"
 "landscape_11.csv"
 ⋮
 "landscape_89.csv"
 "landscape_9.csv"
 "landscape_90.csv"
 "landscape_91.csv"
 "landscape_92.csv"
 "landscape_93.csv"
 "landscape_94.csv"
 "landscape_95.csv"
 "landscape_96.csv"
 "landscape_97.csv"
 "landscape_98.csv"
 "landscape_99.csv"

In [ ]:
function euclid(a::Vector{Float64},b::Vector{Float64})
    
    dist=0
    
    for i in length(a)
        
        dist+=(a[i]-b[i])^2
        
    end
    
    return(sqrt(dist))
end

In [3]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances


Lx=128 
Ly=128
abun=100
nbirth=ndeath=convert(Int64,floor(abun*0.1))
    

#Define dispersal probability as a function of #neighbors for all three dispersal methods
comp_dist=3
maxnb=((2*comp_dist+1)^2)-1
neigh=log.(maxnb,1:maxnb)
comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])   


    result=DataFrame()



    landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",1,".csv"), DataFrame)


    repl=landscape.rep[1] 
    rangepar=landscape.rangepars[1]    

    landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
    landscape[:,:rep]=fill(repl,nrow(landscape))
    landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

    initial=filter(:pres => x-> x .== 1,landscape)
    initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

    init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
    init=Matrix{Int8}(init[:,Not("y")])

    habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
    habmat=Matrix{Float64}(habmat[:,Not("y")])

    #Threshold for a suitable habitat
    mid=quantile(landscape[:,:soiltype],0.5)
    
    d=Normal(0,disps/2)
        
    popmat=deepcopy(init)



    
    

function kill!(A::Matrix{Int8}, B::Matrix{Float64})
    
    samp=findall(popmat.==1)
    A[wsample(samp,habmat[samp],ndeath,replace=false)].=0
    return nothing

end

function recprob(a::CartesianIndex{2},A::Matrix{Int8},disps::Int64,d::Int64,b::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}})
    
    xs=((a[1]-disps):(a[1]+disps))
    xs=xs[0 .<xs .<=Ly]

    ys=(a[2]-disps):(a[2]+disps)
    ys=ys[0 .<ys .<=Lx]
    
    prob=0
   
    if sum(A[xs,ys])>0 
        
        parents=findall(popmat[xs,ys].==1)

        for i in 1:length(parents)
            
            prob+=pdf(Normal(0,disps/2),euclidean([a[1],a[2]],[xs[parents[i][1]],ys[parents[i][2]]]))
        end

        xc=((a[1]-comp_dist):(a[1]+comp_dist))
        xc=xc[0 .<xc .<=Ly]

        yc=(a[2]-comp_dist):(a[2]+comp_dist)
        yc=yc[0 .<yc .<=Lx]
        
        prob*=comp_list[methods][sum(A[xc,yc])+1]
        
        return prob
        
    else
        return prob
    end
    
end


function spawns!(A::Matrix{Int8},dists::Int64,comp_dist::Int64,
        comp_list::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}})  
    
    samp2=shuffle(findall(A.==0))
    
    probs=map(x->recprob(x,A,dists,comp_dist,comp_list),samp2)
    
    A[wsample(samp2,probs,nbirth,replace=false)].=1
    
    return nothing
    
end
    

function sampsim!(A::Matrix{Int8},B::Matrix{Float64},dists::Int64,comp_dist::Int64,
        comp_list::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}}) 
    
    for i in 1:100
        
        kill!(popmat,habmat)
        spawns!(popmat,5,3,comp_list)
        
    end
    
    return nothing
end



#Do the simulation now!






    

 21.022502 seconds (61.94 M allocations: 3.030 GiB, 3.73% gc time, 0.26% compilation time)


In [4]:
@time sampsim!(popmat,habmat,5,3,comp_list)

  8.718863 seconds (56.70 M allocations: 2.786 GiB, 4.81% gc time)


In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances


Lx=128 
Ly=128
abun=100
nbirth=ndeath=convert(Int64,floor(abun*0.1))
    
disps=6
methods=1

#Define dispersal probability as a function of #neighbors for all three dispersal methods
comp_dist=3
maxnb=((2*comp_dist+1)^2)-1
neigh=log.(maxnb,1:maxnb)
comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])   



    result=DataFrame()

    landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",1,".csv"), DataFrame)

    landscape=landscape[landscape.x .<129 .&& landscape.y .<129 ,:]

    repl=landscape.rep[1] 
    rangepar=landscape.rangepars[1]    

    landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
    landscape[:,:rep]=fill(repl,nrow(landscape))
    landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

    initial=filter(:pres => x-> x .== 1,landscape)
    initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

    init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
    init=Matrix(init[:,Not("y")])

    habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
    habmat=Matrix{Float64}(habmat[:,Not("y")])

    #Threshold for a suitable habitat
    mid=quantile(landscape[:,:soiltype],0.5)
    
    d=Normal(0,disps/2)
        
    popmat=deepcopy(init)
    


function sampsim()
    

 for i in 1:50
            
        samp=findall(popmat.==1)
        mort=habmat[samp]
        del=wsample(1:length(mort),mort,ndeath)
        popmat[samp[del]].=0
            
        samp2=shuffle(findall(popmat.==0))

        vals=Float64[]

        for i2 in 1:length(samp2)

            x=samp2[i2][1]
            y=samp2[i2][2]


            xs=((x-disps):(x+disps))
            xs=xs[0 .<xs .<=Ly]

            ys=(y-disps):(y+disps)
            ys=ys[0 .<ys .<=Lx]

            prob=0
            
            if sum(popmat[xs,ys])>0      
                    

                parents=findall(popmat[xs,ys].==1)

                for i2 in 1:length(parents)

                    prob+=pdf(d,euclid([x,y],[xs[parents[i2][1]],ys[parents[i2][2]]]))
                end

                xc=((x-comp_dist):(x+comp_dist))
                xc=xc[0 .<xc .<=Ly]

                yc=(y-comp_dist):(y+comp_dist)
                yc=yc[0 .<yc .<=Lx]
                

                prob*=comp_list[methods][sum(popmat[xc,yc])+1]    

            end

            push!(vals,prob)

        end
    
        popmat[samp2[wsample(1:length(samp2),vals,nbirth)]].=1  
        
    end
    
            popmat1=DataFrame(popmat,:auto)
            rename!(x-> strip(string(x), ['x']),popmat1)
            popmat1[:,:y].=1:Ly
            popmat1=stack(popmat1,1:Lx)
            rename!(popmat1,:variable=> :x, :value => :pres)
            popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
            filter!(:pres => x-> x .== 1,popmat1)
            popmat1[:,:rep].=repl 
            popmat1[:,:rangepar].=rangepar
            popmat1[:,:disp].=disps
            popmat1[:,:method].=methods       
            append!(result,popmat1)

    
end




In [ ]:
function kill!(A::Matrix{Int8}, B::Matrix{Float64})
    
    samp=findall(popmat.==1)
    A[wsample(samp,habmat[samp],ndeath,replace=false)].=0
    return nothing

end

function recprob(a::CartesianIndex{2},A::Matrix{Int8},disps::Int64,d::Int64,b::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}})
    
    xs=((a[1]-disps):(a[1]+disps))
    xs=xs[0 .<xs .<=Ly]

    ys=(a[2]-disps):(a[2]+disps)
    ys=ys[0 .<ys .<=Lx]
    
    prob=0
   
    if sum(A[xs,ys])>0 
        
        parents=findall(popmat[xs,ys].==1)

        for i in 1:length(parents)
            
            prob+=pdf(Normal(0,disps/2),euclidean([a[1],a[2]],[xs[parents[i][1]],ys[parents[i][2]]]))
        end

        xc=((a[1]-comp_dist):(a[1]+comp_dist))
        xc=xc[0 .<xc .<=Ly]

        yc=(a[2]-comp_dist):(a[2]+comp_dist)
        yc=yc[0 .<yc .<=Lx]
        
        prob*=comp_list[methods][sum(A[xc,yc])+1]
        
        return prob
        
    else
        return prob
    end
    
end


function spawns!(A::Matrix{Int8},dists::Int64,comp_dist::Int64,
        comp_list::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}})  
    
    samp2=shuffle(findall(A.==0))
    
    probs=map(x->recprob(x,A,dists,comp_dist,comp_list),samp2)
    
    A[wsample(samp2,probs,nbirth,replace=false)].=1
    
    return nothing
    
end
    

function sampsim!(A::Matrix{Int8},B::Matrix{Float64},dists::Int64,comp_dist::Int64,
        comp_list::Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}}) 
    
    for i in 1:100
        
        kill!(popmat,habmat)
        spawns!(popmat,5,3,comp_list)
        
    end
    
    return nothing
end
    

In [ ]:
@time sampsim!(popmat,habmat,5,3,comp_list)

In [ ]:
sum(popmat)

In [ ]:
   samp=findall(popmat.==1)
    
    wsample(samp,habmat[samp],ndeath,replace=false)
samp

In [ ]:
CSV.write("res1.csv",result)

In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances


Lx=128 
Ly=128
abun=100
nbirth=ndeath=convert(Int64,floor(abun*0.1))
    
#reps=collect(201:224)
disp=[3,4,6,10]
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(disp,methodrefs[:,:ind]))

result=DataFrame()

landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",1,".csv"), DataFrame)

landscape=landscape[landscape.x .<129 .&& landscape.y .<129 ,:]

repl=landscape.rep[1] 
rangepar=landscape.rangepars[1]    

landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
landscape[:,:rep]=fill(repl,nrow(landscape))
landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

initial=filter(:pres => x-> x .== 1,landscape)
initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
init=Matrix(init[:,Not("y")])

habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
habmat=Matrix{Float64}(habmat[:,Not("y")])
    
#Threshold for a suitable habitat
mid=quantile(landscape[:,:soiltype],0.5)
    


for i1 in 1:length(pars)
        
    disps=pars[i1][1]
    methods=pars[i1][2]
        
    #Define dispersal probability as a function of #neighbors for all three dispersal methods
    comp_dist=3
    maxnb=((2*comp_dist+1)^2)-1
    neigh=log.(maxnb,1:maxnb)
    comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])    
        
    d=Normal(0,disps/2)
        
    popmat=deepcopy(init)

    for i in 1:250
            
        samp=findall(popmat.==1)
        mort=habmat[samp]
        del=wsample(1:length(mort),mort,ndeath)
        popmat[samp[del]].=0
            
        samp2=shuffle(findall(popmat.==0))

        vals=Float64[]

        for i2 in 1:length(samp2)

            x=samp2[i2][1]
            y=samp2[i2][2]


            xs=((x-disps):(x+disps))
            xs=xs[0 .<xs .<=Ly]

            ys=(y-disps):(y+disps)
            ys=ys[0 .<ys .<=Lx]

            prob=0
            
            if sum(popmat[xs,ys])>0      
                    

                parents=findall(popmat[xs,ys].==1)

                for i2 in 1:length(parents)

                    prob+=pdf(d,euclidean([x,y],[xs[parents[i2][1]],ys[parents[i2][2]]]))
                end

                xc=((x-comp_dist):(x+comp_dist))
                xc=xc[0 .<xc .<=Ly]

                yc=(y-comp_dist):(y+comp_dist)
                yc=yc[0 .<yc .<=Lx]
                

                prob*=comp_list[methods][sum(popmat[xc,yc])+1]    

            end

            push!(vals,prob)

        end
    
        popmat[samp2[wsample(1:length(samp2),vals,nbirth)]].=1  
        
    end
    
            popmat1=DataFrame(popmat,:auto)
            rename!(x-> strip(string(x), ['x']),popmat1)
            popmat1[:,:y].=1:Ly
            popmat1=stack(popmat1,1:Lx)
            rename!(popmat1,:variable=> :x, :value => :pres)
            popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
            filter!(:pres => x-> x .== 1,popmat1)
            popmat1[:,:rep].=repl 
            popmat1[:,:rangepar].=rangepar
            popmat1[:,:disp].=disps
            popmat1[:,:method].=methodrefs[methods,2]        
            append!(result,popmat1)
        
    
end


In [ ]:

using CSV, DataFrames, Random, LinearAlgebra, Distributions, Distances

Lx=128 
Ly=128
abun=100
timepoints=100
nbirth=ndeath=convert(Int64,floor(abun*0.1))
    
#reps=collect(201:224)
disp=[3,4,6,10]
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(disp,methodrefs[:,:ind]))

lands=[collect(1:5);collect(101:105);collect(201:205)]

for j in 

landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",lands[j],".csv"), DataFrame)

landscape=landscape[landscape.x .<129 .&& landscape.y .<129 ,:]

repl=landscape.rep[1] 
rangepar=landscape.rangepars[1]    

landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
landscape[:,:rep]=fill(repl,nrow(landscape))
landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

initial=filter(:pres => x-> x .== 1,landscape)
initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
init=Matrix(init[:,Not("y")])

habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
habmat=Matrix{Float64}(habmat[:,Not("y")])
    
#Threshold for a suitable habitat
mid=quantile(landscape[:,:soiltype],0.5)
    
result=DataFrame()
    
for i1 in 1:length(pars)
        
    disps=pars[i1][1]
    methods=pars[i1][2]
        
    #Define dispersal probability as a function of #neighbors for all three dispersal methods
    comp_dist=3
    maxnb=((2*comp_dist+1)^2)-1
    neigh=log.(maxnb,1:maxnb)
    comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])    
        
    d=Normal(0,disps/2)
        
    popmat=deepcopy(init)

    for i in 1:250
            
        samp=findall(popmat.==1)
        mort=habmat[samp]
        del=wsample(1:length(mort),mort,ndeath)
        popmat[samp[del]].=0
            
        samp2=shuffle(findall(popmat.==0))

        vals=Float64[]

        for i2 in 1:length(samp2)

            x=samp2[i2][1]
            y=samp2[i2][2]


            xs=((x-disps):(x+disps))
            xs=xs[0 .<xs .<=Ly]

            ys=(y-disps):(y+disps)
            ys=ys[0 .<ys .<=Lx]

            prob=0

            if sum(popmat[xs,ys])>0

                parents=findall(popmat[xs,ys].==1)

                for i2 in 1:length(parents)

                    prob+=pdf(d,euclidean([x,y],[xs[parents[i2][1]],ys[parents[i2][2]]]))
                end

                xc=((x-comp_dist):(x+comp_dist))
                xc=xc[0 .<xc .<=Ly]

                yc=(y-comp_dist):(y+comp_dist)
                yc=yc[0 .<yc .<=Lx]
                

                prob*=comp_list[methods][sum(popmat[xc,yc])+1]    

            end

            push!(vals,prob)

        end
    
        popmat[samp2[wsample(1:length(samp2),vals,nbirth)]].=1  
        
    end
    
            popmat1=DataFrame(popmat,:auto)
            rename!(x-> strip(string(x), ['x']),popmat1)
            popmat1[:,:y].=1:Ly
            popmat1=stack(popmat1,1:Lx)
            rename!(popmat1,:variable=> :x, :value => :pres)
            popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
            filter!(:pres => x-> x .== 1,popmat1)
            popmat1[:,:rep].=repl 
            popmat1[:,:rangepar].=rangepar
            popmat1[:,:disp].=disps
            popmat1[:,:method].=methodrefs[methods,2]        
            append!(result,popmat1)
        
    
end
            


In [ ]:
[collect(1:5);collect(101:105);collect(201:205)]